##### Import the relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error


In [35]:
data = pd.read_csv('/Users/bond/PycharmProjects/Predicting-Player-Salaries/data/Hitters.csv')
df_hitters = data.copy()

In [36]:
df_hitters

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [37]:
# Categorical Variables
print('The League Types are: ', df_hitters['League'].unique())
print('The division Types are: ', df_hitters['Division'].unique())
print('The new league options are: ', df_hitters['NewLeague'].unique())

The League Types are:  ['A' 'N']
The division Types are:  ['E' 'W']
The new league options are:  ['A' 'N']


In [38]:
df_hitters_num = pd.get_dummies(df_hitters, columns=['League', 'Division', 'NewLeague'], drop_first=True)
df_hitters_num

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
0,293,66,1,30,29,14,1,293,66,1,30,29,14,446,33,20,NaN,0,0,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0,1,1,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0,0,1,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0,1,0,1
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,325,9,3,700.0,1,0,1
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,313,381,20,875.0,0,0,0
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,37,113,7,385.0,0,1,0
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,1314,131,12,960.0,0,0,0


In [39]:
df_hitters_num.isnull().sum()

AtBat           0
Hits            0
HmRun           0
Runs            0
RBI             0
Walks           0
Years           0
CAtBat          0
CHits           0
CHmRun          0
CRuns           0
CRBI            0
CWalks          0
PutOuts         0
Assists         0
Errors          0
Salary         59
League_N        0
Division_W      0
NewLeague_N     0
dtype: int64

In [40]:
df_hitters_num_no_null = df_hitters_num.dropna()
df_hitters_num_no_null.isna().sum()

AtBat          0
Hits           0
HmRun          0
Runs           0
RBI            0
Walks          0
Years          0
CAtBat         0
CHits          0
CHmRun         0
CRuns          0
CRBI           0
CWalks         0
PutOuts        0
Assists        0
Errors         0
Salary         0
League_N       0
Division_W     0
NewLeague_N    0
dtype: int64

#### Declare the dependent and Independent variables

In [41]:
X = df_hitters_num_no_null.drop('Salary', axis=1)
y = df_hitters_num_no_null['Salary']

#### Split the data into train and test sets

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

In [43]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Perform Linear Regression

In [44]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [45]:
print("Linear Regression Coefficients are: ", lin_reg.coef_)
print()
print("Linear Regression y-intercept is: ", lin_reg.intercept_)

Linear Regression Coefficients are:  [-205.58645789  395.05565574   89.83392893 -174.8599794  -115.06985424
  173.60350214  -10.56691628 -555.16464857  152.59701271   69.52782305
  537.58546405  278.87679837 -271.53970319   75.51504364   28.46570102
  -25.02896584   24.51588697  -36.02075257  -21.20420211]

Linear Regression y-intercept is:  560.7115294117648


In [46]:
lin_reg_y_pred = lin_reg.predict(X_test)
lin_reg_y_pred

array([1076.40372081,  194.46359798,  208.06762021,  128.35238918,
       1150.00415585,  330.4229848 ,  293.35692767,  374.86435153,
        564.32354921,  503.42424916,  287.14506696,  634.65545388,
       1256.71578437,  162.86177539,  270.4724371 ,  627.03634363,
       1098.78262703,  168.72823705,  635.59519342,  538.2358982 ,
        997.44726273,  453.76638183,  284.64083511, 1020.38051099,
        276.04851337,  421.28103183,  146.57210339,  325.00388094,
        532.0565091 ,  243.25119615,  801.37721717,  101.50348498,
        467.74315444,  548.74006459,  317.74722813,  662.06115499,
        306.60919967,  520.18141935,  584.08586434, 1776.06709074,
        206.74897271,  396.77817827,  150.60653679,  837.80543541,
         18.9969577 , 1169.44216946,  476.86297787,  137.2993431 ,
        895.94365231,  755.83780802,  700.77613482,  412.7776676 ,
          5.93030018,  503.83096087,  586.35600881,  387.73753157,
        273.98893564,  394.6049807 ,  299.42870718,  517.21067

In [47]:
# Compare predictions with actual outputs
lin_comp = pd.DataFrame({'Predicted': lin_reg_y_pred, 'Actual': y_test})
lin_comp

,Predicted,Actual
178,1076.403721,1000.000
296,194.463598,286.667
92,208.067620,70.000
261,128.352389,190.000
120,1150.004156,733.333
...,...,...
307,306.635859,277.500
86,1392.986584,1041.667
142,1004.970762,1300.000
280,396.140794,105.000


In [48]:
print("Linear Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lin_reg_y_pred)))
print("Linear Regression Model Training Score: ", lin_reg.score(X_train, y_train))
print("Linear Regression Model Testing Score: ", lin_reg.score(X_test, y_test))

Linear Regression Model RMSE is:  381.2159334755361
Linear Regression Model Training Score:  0.64547107588673
Linear Regression Model Testing Score:  0.25141497395552603


#### Perform Ridge Regression

In [49]:
CV = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

In [50]:
ridge = RidgeCV(alphas=np.arange(0.1, 10, 0.1), cv=CV, scoring='neg_mean_squared_error')

In [51]:
ridge.fit(X_train, y_train)
ridge_reg_y_pred = ridge.predict(X_test)

print("Ridge Tuning Parameter is: ", (ridge.alpha_))
print()
print("Ridge Model Coefficients: ", (ridge.coef_))
print()
print("Ridge Model Intercept: ", (ridge.intercept_))

Ridge Tuning Parameter is:  1.6

Ridge Model Coefficients:  [-178.74112041  311.07904997   39.06823245 -103.20890733  -60.37443499
  150.78472659  -56.44152446 -178.16607357  128.13537455  134.78799763
  265.14077792  136.49756603 -222.27276972   77.41602604   18.82267603
  -30.96207839   27.90721964  -35.8908812   -29.30753972]

Ridge Model Intercept:  560.7115294117647


In [52]:
print("Ridge Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, ridge_reg_y_pred)))
print("Ridge Regression Model Training Score: ", ridge.score(X_train, y_train))
print("Ridge Regression Model Testing Score: ", ridge.score(X_test, y_test))

Ridge Regression Model RMSE is:  376.78639075240716
Ridge Regression Model Training Score:  0.6359493752558608
Ridge Regression Model Testing Score:  0.2687102890716988


### Perform Lasso Regression

In [53]:
lasso = LassoCV(alphas=np.arange(0.1,10,0.1), cv=CV, tol=1)


In [54]:
lasso.fit(X_train, y_train)
lasso_reg_y_predict = lasso.predict(X_test)

print("Lasso Tuning Parameter is: ", (lasso.alpha_))
print()
print("Lasso Model Coefficients: ", (lasso.coef_))
print()
print("Lasso Model Intercept: ", (lasso.intercept_))

Lasso Tuning Parameter is:  0.30000000000000004

Lasso Model Coefficients:  [134.27849348  48.68809966  25.85560786 -29.83492291  16.69553639
  71.61918683 142.31889864  56.19006024  20.80462501  59.91531326
 -37.27605712  11.2523882  -46.62253601  64.6007483  -42.51241609
 -25.71665384  33.16461687 -27.92254167 -28.9108242 ]

Lasso Model Intercept:  560.7115294117647


In [55]:
print("Lasso Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lasso_reg_y_predict)))
print("Lasso Regression Model Training Score: ", lasso.score(X_train, y_train))
print("Lasso Regression Model Testing Score: ", lasso.score(X_test, y_test))

Lasso Regression Model RMSE is:  401.3585629091625
Lasso Regression Model Training Score:  0.49682638504562004
Lasso Regression Model Testing Score:  0.17021780252903418


#### Compare the Scores

In [56]:
print("Linear Regression Model Training Score: ", lin_reg.score(X_train, y_train))
print("Linear Regression Model Testing Score: ", lin_reg.score(X_test, y_test))
print()
print("Ridge Regression Model Training Score: ", ridge.score(X_train, y_train))
print("Ridge Regression Model Testing Score: ", ridge.score(X_test, y_test))
print()
print("Lasso Regression Model Training Score: ", lasso.score(X_train, y_train))
print("Lasso Regression Model Testing Score: ", lasso.score(X_test, y_test))


Linear Regression Model Training Score:  0.64547107588673
Linear Regression Model Testing Score:  0.25141497395552603

Ridge Regression Model Training Score:  0.6359493752558608
Ridge Regression Model Testing Score:  0.2687102890716988

Lasso Regression Model Training Score:  0.49682638504562004
Lasso Regression Model Testing Score:  0.17021780252903418


In [57]:
#### Compare the root mean squared error
print("Linear Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lin_reg_y_pred)))
print()
print("Lasso Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, lasso_reg_y_predict)))
print()
print("Ridge Regression Model RMSE is: ", math.sqrt(mean_squared_error(y_test, ridge_reg_y_pred)))



Linear Regression Model RMSE is:  381.2159334755361

Lasso Regression Model RMSE is:  401.3585629091625

Ridge Regression Model RMSE is:  376.78639075240716


#### Replacing/Predicting the Missing Values in the Dataframe

In [62]:
df_hitters_nan = df_hitters_num[df_hitters_num['Salary'].isnull()]

In [63]:
df_hitters_nan

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
0,293,66,1,30,29,14,1,293,66,1,30,29,14,446,33,20,NaN,0,0,0
15,183,39,3,20,15,11,3,201,42,3,20,16,11,118,0,0,NaN,0,1,0
18,407,104,6,57,43,65,12,5233,1478,100,643,658,653,912,88,9,NaN,0,1,0
22,22,10,1,4,2,1,6,84,26,2,9,9,3,812,84,11,NaN,0,1,0
30,313,84,9,42,30,39,17,6890,1833,224,1033,864,1087,127,221,7,NaN,0,1,0
32,517,141,27,70,87,52,9,3571,994,215,545,652,337,1378,102,8,NaN,1,1,1
36,161,36,0,19,10,17,4,1053,244,3,156,86,107,70,149,12,NaN,0,0,0
38,346,98,5,31,53,30,16,5913,1615,235,784,901,560,0,0,0,NaN,0,0,0
39,241,61,1,34,12,14,1,241,61,1,34,12,14,166,172,10,NaN,1,1,1
41,216,54,0,21,18,15,18,7318,1926,46,796,627,483,103,84,5,NaN,1,1,1


In [64]:
X_nan = df_hitters_nan.drop('Salary', axis = 1)
y_nan = df_hitters_nan['Salary']

In [65]:
scaler = StandardScaler()
X_nan = scaler.fit_transform(X_nan)

In [67]:
nan_pred = ridge.predict(X_nan)
nan_pred

array([ 233.1513017 ,  182.32271859,  934.96845957,  286.26742702,
        729.9838443 , 1415.00310853,  275.23150279, 1012.62574129,
        245.35843809,  429.55262957,  457.85949005,  405.46196341,
       1563.60406894,  194.63282405,  477.34892303,  137.28454324,
        327.37393507,  772.21355616,  499.09455968,  430.48524149,
       1147.24075604,  277.46103286,  476.75373594,  710.24863961,
        631.76424642,  743.5589759 , 1226.94629255,  243.85085054,
        370.66051298, 1155.91027101,  774.83389724,  335.78068455,
        356.96942711,  276.29029012,  405.50155656,  704.54141726,
        218.71810642,  445.69525501,  366.85207459,  398.87408862,
        240.73952605,  729.12382843,  429.52910972,  524.0706106 ,
        455.83366195,  648.30898639,  400.6855435 , 1258.01305005,
        873.32755227,  192.1181608 ,  215.95422247,  449.24280285,
        382.24242067,  187.47171611,  613.21149333,  267.46888425,
       1326.86181278, 1228.91342585,  380.58703934])

In [68]:
df_nan_full = df_hitters_nan.copy()
df_nan_full['Salary'] = nan_pred
df_nan_full

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
0,293,66,1,30,29,14,1,293,66,1,30,29,14,446,33,20,233.151302,0,0,0
15,183,39,3,20,15,11,3,201,42,3,20,16,11,118,0,0,182.322719,0,1,0
18,407,104,6,57,43,65,12,5233,1478,100,643,658,653,912,88,9,934.968460,0,1,0
22,22,10,1,4,2,1,6,84,26,2,9,9,3,812,84,11,286.267427,0,1,0
30,313,84,9,42,30,39,17,6890,1833,224,1033,864,1087,127,221,7,729.983844,0,1,0
32,517,141,27,70,87,52,9,3571,994,215,545,652,337,1378,102,8,1415.003109,1,1,1
36,161,36,0,19,10,17,4,1053,244,3,156,86,107,70,149,12,275.231503,0,0,0
38,346,98,5,31,53,30,16,5913,1615,235,784,901,560,0,0,0,1012.625741,0,0,0
39,241,61,1,34,12,14,1,241,61,1,34,12,14,166,172,10,245.358438,1,1,1
41,216,54,0,21,18,15,18,7318,1926,46,796,627,483,103,84,5,429.552630,1,1,1
